<a href="https://colab.research.google.com/github/macollipal/challenge2-data-science-LATAM/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

## Cargar dataFrame

importar los datos de la API de Telecom X. Estos datos están disponibles en formato JSON y contienen información esencial sobre los clientes, incluyendo datos demográficos, tipo de servicio contratado y estado de evasión.

In [132]:
import pandas as pd

#df = pd.read_json('https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json')
#df.head()

"""Carga JSON desde URL"""
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"

response = requests.get(url)
data = response.json()
print(f"✅ Datos cargados: {len(data) if isinstance(data, list) else 1} registros")
#data


✅ Datos cargados: 7267 registros


#🔧 Transformación

### **Conozce el conjunto de datos**

Ahora que has extraído los datos, es fundamental comprender la estructura del dataset y el significado de sus columnas. Esta etapa te ayudará a identificar qué variables son más relevantes para el análisis de evasión de clientes.

📌 Para facilitar este proceso, hemos creado un diccionario de datos con la descripción de cada columna. Aunque no es obligatorio utilizarlo, puede ayudarte a comprender mejor la información disponible.
¿Qué debes hacer?

✅ Explorar las columnas del dataset y verificar sus tipos de datos.
✅ Consultar el diccionario para comprender mejor el significado de las variables.
✅ Identificar las columnas más relevantes para el análisis de evasión.


#### **Columnas más importantes para predecir Churn**

Basándome en la información de las columnas que tienes, estas serían las columnas más importantes para predecir Churn, ordenadas por relevancia típica:
Columnas de Alta Importancia

**1. Variables de Contrato y Facturación**

account_contract - Tipo de contrato (mensual vs anual)
account_charges_monthly - Cargos mensuales
account_charges_total - Cargos totales históricos
account_paymentmethod - Método de pago

**2. Variables de Tenure y Demografía**

customer_tenure - Tiempo como cliente (muy importante)
customer_seniorcitizen - Cliente senior
customer_partner - Tiene pareja
customer_dependents - Tiene dependientes

***Columnas de Importancia Media***

**3. Variables de Servicios de Internet**

internet_internetservice - Tipo de servicio de internet
internet_onlinesecurity - Seguridad online
internet_techsupport - Soporte técnico
internet_onlinebackup - Respaldo online

**4. Variables de Servicios de Teléfono**

phone_phoneservice - Servicio telefónico
phone_multiplelines - Múltiples líneas

***Columnas de Menor Importancia***

internet_deviceprotection
internet_streamingtv
internet_streamingmovies
account_paperlessbilling
customer_gender

#### **Columnas clave para empezar tu análisis**
key_features = [
    'customer_tenure',
    'account_contract',
    'account_charges_monthly',
    'account_charges_total',
    'internet_internetservice',
    'customer_seniorcitizen',
    'internet_techsupport',
    'internet_onlinesecurity'
]

### **Comprobación de incoherencias en los datos**
En este paso, verifica si hay problemas en los datos que puedan afectar el análisis. Presta atención a valores ausentes, duplicados, errores de formato e inconsistencias en las categorías. Este proceso es esencial para asegurarte de que los datos estén listos para las siguientes etapas.
📌 Tips:

🔗 Documentación de pandas.unique()
🔗 Documentación de pandas.Series.dt.normalize()

In [133]:
"""Normaliza JSON anidado a DataFrame"""
if isinstance(data, list):
    df = json_normalize(data)
else:
    df = json_normalize([data])

# Limpiar nombres de columnas
df.columns = df.columns.str.replace('.', '_', regex=False)
df.columns = df.columns.str.lower()

print(f"✅ DataFrame creado: {df.shape[0]} filas × {df.shape[1]} columnas")
display(df.head(3))

✅ DataFrame creado: 7267 filas × 21 columnas


,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_onlinebackup,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges_monthly,account_charges_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


In [134]:
df['customerid'].unique()

array(['0002-ORFBO', '0003-MKNFE', '0004-TLHLJ', ..., '9992-UJOEL',
       '9993-LHIEB', '9995-HOTOH'], dtype=object)

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7267 non-null   object 
 1   churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_seniorcitizen     7267 non-null   int64  
 4   customer_partner           7267 non-null   object 
 5   customer_dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_phoneservice         7267 non-null   object 
 8   phone_multiplelines        7267 non-null   object 
 9   internet_internetservice   7267 non-null   object 
 10  internet_onlinesecurity    7267 non-null   object 
 11  internet_onlinebackup      7267 non-null   object 
 12  internet_deviceprotection  7267 non-null   object 
 13  internet_techsupport       7267 non-null   objec

In [136]:
# prompt: mostrar cuantos registros son nulos o vacios en la columna churn

# Calculate the number of null or empty values in the 'churn' column
null_or_empty_churn = df['churn'].isnull().sum() + (df['churn'] == '').sum() + (df['churn'] == 0).sum()

print(f"Número de registros nulos o vacíos en la columna 'churn': {null_or_empty_churn}")

# Display the current state of the DataFrame info after previous filtering
#df.info()

Número de registros nulos o vacíos en la columna 'churn': 224


In [137]:
# prompt: eliminar nulos cuando la columna churn este vacia o nula 0 ""

import numpy as np

df = df[df['churn'].notna()]
df = df[df['churn'] != '']
df = df[df['churn'] != 0]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7043 non-null   object 
 1   churn                      7043 non-null   object 
 2   customer_gender            7043 non-null   object 
 3   customer_seniorcitizen     7043 non-null   int64  
 4   customer_partner           7043 non-null   object 
 5   customer_dependents        7043 non-null   object 
 6   customer_tenure            7043 non-null   int64  
 7   phone_phoneservice         7043 non-null   object 
 8   phone_multiplelines        7043 non-null   object 
 9   internet_internetservice   7043 non-null   object 
 10  internet_onlinesecurity    7043 non-null   object 
 11  internet_onlinebackup      7043 non-null   object 
 12  internet_deviceprotection  7043 non-null   object 
 13  internet_techsupport       7043 non-null   object 
 1

In [138]:
# prompt: en el dtype sigue apareciendo tiypo object la comuna crurn

df['churn'] = df['churn'].astype(str)
#df.info()


In [139]:
# prompt: customer_seniorcitizen cambiar el 0 por un NO y 1 Yes

df['customer_seniorcitizen'] = df['customer_seniorcitizen'].replace({'0': 'No', '1': 'Yes'})

# Verificar el cambio
print("\nValores únicos en 'customer_seniorcitizen' después del reemplazo:")
display(df['customer_seniorcitizen'].value_counts())

print("\nPrimeras filas con la columna modificada:")
#display(df.head())


Valores únicos en 'customer_seniorcitizen' después del reemplazo:


,count
customer_seniorcitizen,
0,5901
1,1142



Primeras filas con la columna modificada:


### **Manejo de inconsistencias**
 Aplicar las correcciones necesarias. Ajusta los datos para asegurarte de que estén completos y coherentes, preparándolos para las siguientes etapas del análisis.

In [140]:
# prompt: account_charges_total a float64

import pandas as pd
df['account_charges_total'] = pd.to_numeric(df['account_charges_total'], errors='coerce')
df['account_charges_total'] = df['account_charges_total'].astype('float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7043 non-null   object 
 1   churn                      7043 non-null   object 
 2   customer_gender            7043 non-null   object 
 3   customer_seniorcitizen     7043 non-null   int64  
 4   customer_partner           7043 non-null   object 
 5   customer_dependents        7043 non-null   object 
 6   customer_tenure            7043 non-null   int64  
 7   phone_phoneservice         7043 non-null   object 
 8   phone_multiplelines        7043 non-null   object 
 9   internet_internetservice   7043 non-null   object 
 10  internet_onlinesecurity    7043 non-null   object 
 11  internet_onlinebackup      7043 non-null   object 
 12  internet_deviceprotection  7043 non-null   object 
 13  internet_techsupport       7043 non-null   object 
 1

### **Estandarización y transformación de datos (opcional)**
La estandarización y transformación de datos es una etapa opcional, pero altamente recomendada, ya que busca hacer que la información sea más consistente, comprensible y adecuada para el análisis. Durante esta fase, por ejemplo, puedes convertir valores textuales como "Sí" y "No" en valores binarios (1 y 0), lo que facilita el procesamiento matemático y la aplicación de modelos analíticos.
Además, traducir o renombrar columnas y datos hace que la información sea más accesible y fácil de entender, especialmente cuando se trabaja con fuentes externas o términos técnicos. Aunque no es un paso obligatorio, puede mejorar significativamente la claridad y comunicación de los resultados, facilitando la interpretación y evitando confusiones, especialmente al compartir información con stakeholders no técnicos.

In [129]:
# prompt: convertir los valores textuales "Yes", "No" en 1 y 0, sino dejar el mismo

# Función para convertir 'Yes'/'No' a 1/0 o mantener el valor original
def convert_yes_no(value):
  if value == 'Yes':
    return 1
  elif value == 'No':
    return 0
  else:
    return value

# Aplicar la función a las columnas relevantes
df['customer_seniorcitizen'] = df['customer_seniorcitizen'].apply(convert_yes_no)
df['churn'] = df['churn'].apply(convert_yes_no) # Asumiendo que 'churn' también tiene 'Yes'/'No'
df['account_paperlessbilling'] = df['account_paperlessbilling'].apply(convert_yes_no) # Asumiendo que 'account_paperlessbilling' tiene 'Yes'/'No'

# Aplicar la función a las columnas relacionadas con servicios de internet (asumiendo 'Yes'/'No' o 'No internet service')
internet_cols = [
    'internet_onlinesecurity',
    'internet_onlinebackup',
    'internet_deviceprotection',
    'internet_techsupport',
    'internet_streamingtv',
    'internet_streamingmovies'
]

for col in internet_cols:
    df[col] = df[col].apply(convert_yes_no)

# Aplicar la función a las columnas relacionadas con servicios de teléfono (asumiendo 'Yes'/'No' o 'No phone service')
phone_cols = [
    'phone_phoneservice',
    'phone_multiplelines'
]

for col in phone_cols:
     df[col] = df[col].apply(convert_yes_no)

print("\nDataFrame después de convertir 'Yes'/'No' a 1/0:")
display(df.head())
df.info()


DataFrame después de convertir 'Yes'/'No' a 1/0:


,customerid,churn,customer_gender,customer_seniorcitizen,customer_partner,customer_dependents,customer_tenure,phone_phoneservice,phone_multiplelines,internet_internetservice,...,internet_deviceprotection,internet_techsupport,internet_streamingtv,internet_streamingmovies,account_contract,account_paperlessbilling,account_paymentmethod,account_charges_monthly,account_charges_total,account_charges_daily
0,0002-ORFBO,0,Female,0,Yes,Yes,9,1,0,DSL,...,0,1,1,0,One year,1,Mailed check,65.6,593.30,2.19
1,0003-MKNFE,0,Male,0,No,No,9,1,1,DSL,...,0,0,0,1,Month-to-month,0,Mailed check,59.9,542.40,2.00
2,0004-TLHLJ,1,Male,0,No,No,4,1,0,Fiber optic,...,1,0,0,0,Month-to-month,1,Electronic check,73.9,280.85,2.46
3,0011-IGKFF,1,Male,1,Yes,No,13,1,0,Fiber optic,...,1,0,1,1,Month-to-month,1,Electronic check,98.0,1237.85,3.27
4,0013-EXCHZ,1,Female,1,Yes,No,3,1,0,Fiber optic,...,0,1,1,0,Month-to-month,1,Mailed check,83.9,267.40,2.80


<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7043 non-null   object 
 1   churn                      7043 non-null   int64  
 2   customer_gender            7043 non-null   object 
 3   customer_seniorcitizen     7043 non-null   int64  
 4   customer_partner           7043 non-null   object 
 5   customer_dependents        7043 non-null   object 
 6   customer_tenure            7043 non-null   int64  
 7   phone_phoneservice         7043 non-null   int64  
 8   phone_multiplelines        7043 non-null   object 
 9   internet_internetservice   7043 non-null   object 
 10  internet_onlinesecurity    7043 non-null   object 
 11  internet_onlinebackup      7043 non-null   object 
 12  internet_deviceprotection  7043 non-null   object 
 13  internet_techsupport       7043 non-null   object 
 1

### **Columna de cuentas diarias**
Ahora que los datos están limpios, es momento de crear la columna **"Cuentas_Diarias"**. Utiliza la facturación mensual para calcular el valor diario, proporcionando una visión más detallada del comportamiento de los clientes a lo largo del tiempo.**Esta columna te ayudará a profundizar en el análisis y a obtener información valiosa para las siguientes etapas.**

In [141]:
# prompt: Utiliza la facturación mensual para calcular el valor diario, solo 2 decimales, proporcionando una visión más detallada del comportamiento de los clientes a lo largo del tiempo.

# Asumimos que el mes tiene 30 días para un cálculo diario aproximado.
# Calcula el valor diario dividiendo los cargos mensuales por 30.
# Redondea a 2 decimales para mantener precisión financiera.
df['account_charges_daily'] = (df['account_charges_monthly'] / 30).round(2)

print("\nPrimeras 5 filas con la nueva columna 'account_charges_daily':")
display(df[['account_charges_monthly', 'account_charges_daily']].head())

print("\nInformación del DataFrame después de añadir 'account_charges_daily':")
df.info()


Primeras 5 filas con la nueva columna 'account_charges_daily':


,account_charges_monthly,account_charges_daily
0,65.6,2.19
1,59.9,2.00
2,73.9,2.46
3,98.0,3.27
4,83.9,2.80



Información del DataFrame después de añadir 'account_charges_daily':
<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 7043 non-null   object 
 1   churn                      7043 non-null   object 
 2   customer_gender            7043 non-null   object 
 3   customer_seniorcitizen     7043 non-null   int64  
 4   customer_partner           7043 non-null   object 
 5   customer_dependents        7043 non-null   object 
 6   customer_tenure            7043 non-null   int64  
 7   phone_phoneservice         7043 non-null   object 
 8   phone_multiplelines        7043 non-null   object 
 9   internet_internetservice   7043 non-null   object 
 10  internet_onlinesecurity    7043 non-null   object 
 11  internet_onlinebackup      7043 non-null   object 
 12  internet_deviceprotection  7043 non-nul

#📊 Carga y análisis

#📄Informe final

In [142]:
# Guardar CSV (opcional)
df.to_csv('datos_normalizados.csv', index=False, decimal=',',sep=';', mode='w')
#print(f"\n💾 Archivo guardado: datos_normalizados.csv")

# Para descargar en Colab
from google.colab import files
files.download('datos_normalizados.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>